# Chapter 49: Edge Computing

Edge computing moves computation from centralized servers to geographically distributed locations, reducing latency by executing code closer to users. Next.js leverages the Edge Runtime—a lightweight V8 isolate environment—to execute Middleware, Route Handlers, and rendering at 100+ global locations. This chapter explores architectural patterns for edge-native applications, runtime differences from Node.js, and strategies for data access at the edge.

By the end of this chapter, you'll master the Edge Runtime's constraints and capabilities, implement zero-cold-start middleware for authentication and A/B testing, architect globally distributed caching strategies, access databases from edge locations with connection pooling, handle geographically-aware routing and localization, and optimize performance using edge-native patterns over traditional serverless functions.

## 49.1 Understanding Edge Computing

The architectural shift from regional to global compute deployment.

### Edge vs Node.js Runtime Comparison

Understanding the fundamental differences between execution environments:

```typescript
// Runtime characteristics comparison

/**
 * NODE.JS RUNTIME (Traditional)
 * - Full Node.js API access (fs, crypto, http)
 * - Native module support (C++ addons)
 * - Longer cold starts (100-1000ms)
 * - Regional deployment (single region)
 * - Stateful connections (WebSockets, SSE)
 * - Larger bundle sizes accepted
 */

/**
 * EDGE RUNTIME (V8 Isolates)
 * - Lightweight JavaScript engine
 * - Sub-millisecond cold starts
 * - Global deployment (100+ locations)
 * - Zero-maintenance scaling
 * - Limited API surface (no Node.js modules)
 * - Streaming SSR support
 * - Request/Response API only
 */

// Configuration in Next.js
// next.config.js
module.exports = {
  experimental: {
    runtime: 'experimental-edge', // For specific routes
  },
};
```

### Edge Runtime Constraints and APIs

Available APIs in the Edge Runtime:

```typescript
// app/api/edge-example/route.ts
export const runtime = 'edge'; // Enable Edge Runtime

export async function GET(request: Request) {
  // ✅ Available APIs:
  
  // Standard Web APIs
  const url = new URL(request.url);
  const headers = new Headers(request.headers);
  const params = new URLSearchParams(url.search);
  
  // Fetch (with keep-alive)
  const data = await fetch('https://api.example.com', {
    next: { revalidate: 60 },
  });
  
  // Web Crypto API
  const encoder = new TextEncoder();
  const data_to_hash = encoder.encode('password');
  const hash = await crypto.subtle.digest('SHA-256', data_to_hash);
  
  // Readable/Writable Streams
  const stream = new ReadableStream({
    start(controller) {
      controller.enqueue(new TextEncoder().encode('Hello'));
      controller.close();
    },
  });
  
  // Intl (Internationalization)
  const formatter = new Intl.DateTimeFormat('en-US', {
    timeZone: 'America/New_York',
  });
  
  // ❌ Not available:
  // - fs (file system)
  // - process.env (use global env instead)
  // - Node.js streams (use Web Streams)
  // - Native modules (sharp, bcrypt, etc.)
  
  return new Response(stream, {
    headers: {
      'Content-Type': 'text/plain',
      'Cache-Control': 'public, max-age=60',
    },
  });
}
```

### Geographical Distribution Architecture

How Vercel's edge network routes requests:

```typescript
// Understanding request flow:
// 1. User in Tokyo requests your-site.com
// 2. DNS resolves to nearest Vercel Edge Location (Tokyo)
// 3. Middleware executes in Tokyo (hono1)
// 4. Route Handler executes in Tokyo (if edge runtime)
// 5. Static assets served from Tokyo CDN cache
// 6. Dynamic data fetched from origin (if needed)

// Accessing geographic information
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  // Vercel-specific headers (available at edge)
  const country = request.geo?.country; // 'US', 'JP', etc.
  const city = request.geo?.city; // 'San Francisco'
  const region = request.geo?.region; // 'California'
  const latitude = request.geo?.latitude;
  const longitude = request.geo?.longitude;
  
  // IP address
  const ip = request.ip || '127.0.0.1';
  
  // Edge runtime identification
  const runtime = process.env.NEXT_RUNTIME; // 'edge' or 'nodejs'
  
  return NextResponse.next({
    headers: {
      'x-user-country': country || 'unknown',
      'x-user-city': city || 'unknown',
    },
  });
}
```

## 49.2 Edge Functions

Lightweight compute for API routes at the edge.

### Edge Route Handlers

Building API endpoints with minimal latency:

```typescript
// app/api/geolocation/route.ts
export const runtime = 'edge';
export const preferredRegion = 'iad1'; // Optional: prefer US East

export async function GET(request: Request) {
  // Get client IP and geo data
  const ip = request.headers.get('x-forwarded-for') || 'unknown';
  
  // GeoIP lookup at edge (using Vercel's built-in or external)
  const country = request.headers.get('cf-ipcountry') || 
                  request.headers.get('x-vercel-ip-country');
  
  // Currency and locale based on country
  const localizationMap: Record<string, { currency: string; locale: string }> = {
    US: { currency: 'USD', locale: 'en-US' },
    GB: { currency: 'GBP', locale: 'en-GB' },
    JP: { currency: 'JPY', locale: 'ja-JP' },
    DE: { currency: 'EUR', locale: 'de-DE' },
  };
  
  const localization = localizationMap[country || 'US'] || localizationMap.US;
  
  return Response.json({
    ip,
    country,
    ...localization,
    timestamp: new Date().toISOString(),
    edgeRegion: process.env.VERCEL_REGION || 'unknown',
  });
}

// app/api/weather/[city]/route.ts
export const runtime = 'edge';

interface WeatherData {
  temp: number;
  condition: string;
  humidity: number;
}

// Cache weather data at edge for 5 minutes
export const revalidate = 300;

export async function GET(
  request: Request,
  { params }: { params: { city: string } }
) {
  const city = decodeURIComponent(params.city);
  
  // Check edge cache first (using Cache API)
  const cacheKey = new Request(request.url);
  const cache = caches.default;
  let response = await cache.match(cacheKey);
  
  if (!response) {
    // Fetch from weather API
    const weatherRes = await fetch(
      `https://api.weather.com/v1/current?city=${encodeURIComponent(city)}`,
      {
        headers: { Authorization: `Bearer ${process.env.WEATHER_API_KEY}` },
      }
    );
    
    const data: WeatherData = await weatherRes.json();
    
    response = new Response(JSON.stringify(data), {
      headers: {
        'Content-Type': 'application/json',
        'Cache-Control': 'public, max-age=300',
      },
    });
    
    // Store in edge cache
    await cache.put(cacheKey, response.clone());
  }
  
  return response;
}
```

### Edge Database Access

Connecting to databases from edge locations:

```typescript
// lib/db-edge.ts
// Using connection poolers compatible with edge (HTTP-based)

import { createClient } from '@vercel/postgres'; // Edge-compatible
import { drizzle } from 'drizzle-orm/vercel-postgres';
import { sql } from '@vercel/postgres';

// Option 1: Vercel Postgres (built for edge)
export const db = drizzle(sql);

// Option 2: Neon Serverless Driver (HTTP-based)
import { Pool } from '@neondatabase/serverless';
import { PrismaNeon } from '@prisma/adapter-neon';
import { PrismaClient } from '@prisma/client';

const pool = new Pool({ connectionString: process.env.DATABASE_URL });
const adapter = new PrismaNeon(pool);
export const prismaEdge = new PrismaClient({ adapter });

// Option 3: PlanetScale (HTTP-based)
import { connect } from '@planetscale/database';
const conn = connect({
  host: process.env.DATABASE_HOST,
  username: process.env.DATABASE_USERNAME,
  password: process.env.DATABASE_PASSWORD,
});

// Edge API route using database
// app/api/users/[id]/route.ts
export const runtime = 'edge';

export async function GET(
  request: Request,
  { params }: { params: { id: string } }
) {
  try {
    // Query using Vercel Postgres
    const { rows } = await sql`
      SELECT id, name, email, created_at 
      FROM users 
      WHERE id = ${params.id}
    `;
    
    if (rows.length === 0) {
      return new Response('Not found', { status: 404 });
    }
    
    return Response.json(rows[0]);
  } catch (error) {
    return new Response('Database error', { status: 500 });
  }
}
```

### AI Inference at the Edge

Running machine learning models close to users:

```typescript
// app/api/ai/translate/route.ts
export const runtime = 'edge';

export async function POST(request: Request) {
  const { text, targetLang } = await request.json();
  
  // Using Cloudflare AI or OpenAI at edge
  const response = await fetch(
    'https://api.cloudflare.com/client/v4/ai/run/@cf/meta/m2m100-1.2b',
    {
      method: 'POST',
      headers: {
        'Authorization': `Bearer ${process.env.CF_API_TOKEN}`,
        'Content-Type': 'application/json',
      },
      body: JSON.stringify({
        text,
        target_lang: targetLang,
      }),
    }
  );
  
  const result = await response.json();
  
  return Response.json({
    translation: result.result.translated_text,
    processedAt: process.env.VERCEL_REGION,
  });
}

// Streaming AI responses (OpenAI/Anthropic)
// app/api/ai/chat/route.ts
export const runtime = 'edge';

export async function POST(request: Request) {
  const { messages } = await request.json();
  
  const stream = await fetch('https://api.openai.com/v1/chat/completions', {
    method: 'POST',
    headers: {
      'Content-Type': 'application/json',
      'Authorization': `Bearer ${process.env.OPENAI_API_KEY}`,
    },
    body: JSON.stringify({
      model: 'gpt-4',
      messages,
      stream: true,
    }),
  });
  
  // Transform and stream response
  return new Response(stream.body, {
    headers: {
      'Content-Type': 'text/event-stream',
      'Cache-Control': 'no-cache',
    },
  });
}
```

## 49.3 Edge Middleware

Running code before requests reach your application.

### Authentication at the Edge

Zero-latency session validation:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';
import { jwtVerify } from 'jose'; // Edge-compatible JWT library

export const config = {
  matcher: ['/dashboard/:path*', '/api/protected/:path*'],
};

export async function middleware(request: NextRequest) {
  const token = request.cookies.get('token')?.value;
  
  if (!token) {
    return NextResponse.redirect(new URL('/login', request.url));
  }
  
  try {
    // Verify JWT at the edge (no database lookup)
    const secret = new TextEncoder().encode(process.env.JWT_SECRET);
    const { payload } = await jwtVerify(token, secret);
    
    // Add user info to headers for downstream use
    const requestHeaders = new Headers(request.headers);
    requestHeaders.set('x-user-id', payload.sub as string);
    requestHeaders.set('x-user-role', payload.role as string);
    
    return NextResponse.next({
      request: { headers: requestHeaders },
    });
  } catch (error) {
    // Invalid token
    return NextResponse.redirect(new URL('/login', request.url));
  }
}

// Rate limiting at edge
// middleware.ts (alternative)
import { Ratelimit } from '@upstash/ratelimit';
import { Redis } from '@upstash/redis';

const ratelimit = new Ratelimit({
  redis: Redis.fromEnv(),
  limiter: Ratelimit.slidingWindow(10, '10 s'),
});

export async function middleware(request: NextRequest) {
  const ip = request.ip ?? '127.0.0.1';
  
  const { success, limit, reset, remaining } = await ratelimit.limit(ip);
  
  if (!success) {
    return new NextResponse('Too Many Requests', {
      status: 429,
      headers: {
        'X-RateLimit-Limit': limit.toString(),
        'X-RateLimit-Remaining': remaining.toString(),
        'X-RateLimit-Reset': reset.toString(),
      },
    });
  }
  
  return NextResponse.next();
}
```

### Geographical Routing

Location-based content delivery:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  const country = request.geo?.country || 'US';
  
  // Redirect EU users to GDPR-compliant privacy policy
  if (pathname === '/privacy' && isEUCountry(country)) {
    return NextResponse.rewrite(new URL('/privacy/eu', request.url));
  }
  
  // Block countries under sanctions
  if (isSanctionedCountry(country)) {
    return new NextResponse('Access Denied', { status: 403 });
  }
  
  // Currency detection for e-commerce
  if (pathname.startsWith('/shop')) {
    const currency = getCurrencyByCountry(country);
    const response = NextResponse.next();
    response.cookies.set('preferred-currency', currency, {
      maxAge: 60 * 60 * 24 * 30, // 30 days
    });
    return response;
  }
  
  // Language redirection
  if (pathname === '/') {
    const acceptLang = request.headers.get('accept-language') || '';
    const preferredLang = parseAcceptLanguage(acceptLang);
    
    if (preferredLang !== 'en' && !pathname.startsWith(`/${preferredLang}`)) {
      return NextResponse.redirect(
        new URL(`/${preferredLang}${pathname}`, request.url)
      );
    }
  }
  
  return NextResponse.next();
}

function isEUCountry(country: string): boolean {
  const euCountries = ['DE', 'FR', 'IT', 'ES', 'NL', 'BE', 'AT', 'IE', 'DK', 'FI', 'SE', 'PT', 'GR', 'PL', 'CZ', 'HU', 'SK', 'SI', 'EE', 'LV', 'LT', 'LU', 'MT', 'CY', 'HR', 'BG', 'RO'];
  return euCountries.includes(country);
}

function getCurrencyByCountry(country: string): string {
  const map: Record<string, string> = {
    US: 'USD', GB: 'GBP', JP: 'JPY', EU: 'EUR',
    CA: 'CAD', AU: 'AUD', CH: 'CHF', SE: 'SEK',
  };
  return map[country] || 'USD';
}

// Config to run middleware on specific paths
export const config = {
  matcher: [
    '/((?!_next/static|_next/image|favicon.ico|api/auth).*)',
  ],
};
```

### A/B Testing at Edge

Zero-flicker experiment assignment:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';
import { createHash } from 'crypto';

export function middleware(request: NextRequest) {
  const cookie = request.cookies.get('experiment-id');
  let experimentId = cookie?.value;
  
  // Generate consistent experiment ID based on user IP or cookie
  if (!experimentId) {
    const identifier = request.ip || Math.random().toString();
    experimentId = createHash('md5')
      .update(identifier)
      .digest('hex')
      .substring(0, 8);
  }
  
  // Determine variant (50/50 split)
  const variant = parseInt(experimentId, 16) % 2 === 0 ? 'control' : 'treatment';
  
  // Rewrite to variant path
  const url = request.nextUrl.clone();
  
  if (url.pathname === '/landing') {
    url.pathname = `/landing/${variant}`;
    
    const response = NextResponse.rewrite(url);
    
    // Set cookie if not present
    if (!cookie) {
      response.cookies.set('experiment-id', experimentId, {
        maxAge: 60 * 60 * 24 * 30, // 30 days
        httpOnly: true,
        secure: true,
      });
    }
    
    // Add headers for analytics
    response.headers.set('x-experiment-variant', variant);
    response.headers.set('x-experiment-id', experimentId);
    
    return response;
  }
  
  return NextResponse.next();
}
```

## 49.4 Edge Caching

Global content distribution strategies.

### Cache API Usage

Programmatic caching at the edge:

```typescript
// lib/cache/edge.ts
export const EDGE_CACHE_TTL = {
  STATIC: 86400,    // 24 hours
  DYNAMIC: 60,      // 1 minute
  API: 300,         // 5 minutes
  USER: 0,          // No cache
};

export async function getCachedOrFetch(
  request: Request,
  fetcher: () => Promise<Response>,
  ttl: number = EDGE_CACHE_TTL.DYNAMIC
): Promise<Response> {
  const cache = caches.default;
  const cacheKey = new Request(request.url, request);
  
  // Try cache first
  let response = await cache.match(cacheKey);
  
  if (!response) {
    // Fetch from origin
    response = await fetcher();
    
    // Clone for cache storage
    const cacheResponse = response.clone();
    
    // Only cache successful responses
    if (response.status === 200 && ttl > 0) {
      // Add cache headers
      cacheResponse.headers.set('Cache-Control', `public, max-age=${ttl}`);
      await cache.put(cacheKey, cacheResponse);
    }
  }
  
  return response;
}

// Usage in API route
// app/api/products/[id]/route.ts
export const runtime = 'edge';

export async function GET(
  request: Request,
  { params }: { params: { id: string } }
) {
  return getCachedOrFetch(
    request,
    async () => {
      const product = await fetchFromDatabase(params.id);
      return Response.json(product);
    },
    EDGE_CACHE_TTL.API
  );
}
```

### Stale-While-Revalidate at Edge

Serving cached content while refreshing in background:

```typescript
// middleware/cache-strategy.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export async function middleware(request: NextRequest) {
  const response = NextResponse.next();
  
  // Apply different caching strategies based on path
  const { pathname } = request.nextUrl;
  
  if (pathname.startsWith('/api/public')) {
    // SWR pattern: serve stale for 1 hour, revalidate in background after 1 minute
    response.headers.set(
      'Cache-Control',
      'public, s-maxage=60, stale-while-revalidate=3600'
    );
  } else if (pathname.startsWith('/api/user')) {
    // Private data - no caching
    response.headers.set(
      'Cache-Control',
      'private, no-cache, no-store, max-age=0'
    );
  } else if (pathname.startsWith('/api/static')) {
    // Immutable assets
    response.headers.set(
      'Cache-Control',
      'public, max-age=31536000, immutable'
    );
  }
  
  return response;
}

// Tag-based invalidation
// app/api/revalidate/route.ts
export const runtime = 'edge';

export async function POST(request: Request) {
  const { tag, secret } = await request.json();
  
  if (secret !== process.env.REVALIDATE_SECRET) {
    return new Response('Unauthorized', { status: 401 });
  }
  
  // Purge cache by tag (using Cloudflare or Vercel API)
  const purgeRes = await fetch(
    `https://api.cloudflare.com/client/v4/zones/${process.env.CF_ZONE_ID}/purge_cache`,
    {
      method: 'POST',
      headers: {
        'Authorization': `Bearer ${process.env.CF_API_TOKEN}`,
        'Content-Type': 'application/json',
      },
      body: JSON.stringify({ tags: [tag] }),
    }
  );
  
  return Response.json({ 
    success: purgeRes.ok,
    tag,
    timestamp: new Date().toISOString(),
  });
}
```

## 49.5 Global Deployment Patterns

Architecting for worldwide performance.

### Regional Data Residency

Handling data sovereignty requirements:

```typescript
// middleware/regional.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

// Map countries to regional data centers
const REGIONAL_ROUTES: Record<string, string> = {
  DE: 'eu-central-1', // Germany -> Frankfurt
  FR: 'eu-west-3',    // France -> Paris
  GB: 'eu-west-2',    // UK -> London
  JP: 'ap-northeast-1', // Japan -> Tokyo
  AU: 'ap-southeast-2', // Australia -> Sydney
  US: 'us-east-1',    // Default US
};

export function middleware(request: NextRequest) {
  const country = request.geo?.country || 'US';
  const region = REGIONAL_ROUTES[country] || 'us-east-1';
  
  // Route to regional API endpoint
  const url = request.nextUrl.clone();
  
  if (url.pathname.startsWith('/api/data')) {
    // Rewrite to regional subdomain or path
    url.hostname = `${region}.api.yourservice.com`;
    return NextResponse.rewrite(url);
  }
  
  // Add region header for application logic
  const response = NextResponse.next();
  response.headers.set('x-preferred-region', region);
  response.headers.set('x-user-country', country);
  
  return response;
}

// Client-side regional awareness
// hooks/use-region.ts
'use client';

import { useEffect, useState } from 'react';

interface RegionInfo {
  country: string;
  region: string;
  latency: number;
}

export function useRegion() {
  const [region, setRegion] = useState<RegionInfo | null>(null);
  
  useEffect(() => {
    // Measure latency to different endpoints
    const measureLatency = async (endpoint: string): Promise<number> => {
      const start = performance.now();
      await fetch(endpoint, { method: 'HEAD', cache: 'no-store' });
      return performance.now() - start;
    };
    
    // Determine best region based on latency
    const detectRegion = async () => {
      const regions = [
        { id: 'us-east', url: 'https://us.api.example.com/ping' },
        { id: 'eu-west', url: 'https://eu.api.example.com/ping' },
        { id: 'ap-south', url: 'https://ap.api.example.com/ping' },
      ];
      
      const measurements = await Promise.all(
        regions.map(async (r) => ({
          ...r,
          latency: await measureLatency(r.url),
        }))
      );
      
      const best = measurements.reduce((prev, curr) => 
        curr.latency < prev.latency ? curr : prev
      );
      
      setRegion({
        country: 'detected',
        region: best.id,
        latency: best.latency,
      });
    };
    
    detectRegion();
  }, []);
  
  return region;
}
```

### Multi-Region Database Strategies

Read replicas and write routing:

```typescript
// lib/db/multi-region.ts
interface DatabaseConfig {
  write: string;
  read: string[];
  region: string;
}

const DB_CONFIGS: Record<string, DatabaseConfig> = {
  'us-east-1': {
    write: process.env.DATABASE_URL_US!,
    read: [
      process.env.DATABASE_READ_URL_US_EAST!,
      process.env.DATABASE_READ_URL_US_WEST!,
    ],
    region: 'us-east-1',
  },
  'eu-west-1': {
    write: process.env.DATABASE_URL_EU!,
    read: [
      process.env.DATABASE_READ_URL_EU_WEST!,
      process.env.DATABASE_READ_URL_EU_NORTH!,
    ],
    region: 'eu-west-1',
  },
};

// Edge function with regional database selection
// app/api/data/regional/route.ts
export const runtime = 'edge';

export async function GET(request: Request) {
  // Get preferred region from header or geo
  const region = request.headers.get('x-preferred-region') || 'us-east-1';
  const config = DB_CONFIGS[region] || DB_CONFIGS['us-east-1'];
  
  // Connect to nearest read replica
  const db = createConnection(config.read[0]);
  
  const data = await db.query('SELECT * FROM content WHERE region = $1', [region]);
  
  return Response.json({
    data,
    servedFrom: config.region,
    timestamp: new Date().toISOString(),
  });
}

// Write operations always go to primary
export async function POST(request: Request) {
  const region = request.headers.get('x-preferred-region') || 'us-east-1';
  const config = DB_CONFIGS[region] || DB_CONFIGS['us-east-1'];
  
  // Write to primary (may have higher latency)
  const db = createConnection(config.write);
  
  const body = await request.json();
  const result = await db.query(
    'INSERT INTO content (data, region) VALUES ($1, $2) RETURNING *',
    [body, region]
  );
  
  // Replicate to read replicas asynchronously
  
  return Response.json({
    data: result[0],
    writtenTo: config.region,
  });
}
```

## 49.6 Performance Considerations

Optimizing for edge-specific constraints.

### Bundle Size Optimization

Minimizing cold start impact (though edge has near-zero cold starts, bundle size affects memory):

```typescript
// Optimize imports for edge
// ❌ Bad: Import entire library
import _ from 'lodash';
const result = _.pick(data, ['id', 'name']);

// ✅ Good: Import specific function (tree-shakable)
import pick from 'lodash/pick';
const result = pick(data, ['id', 'name']);

// ✅ Better: Use native alternatives
const result = Object.fromEntries(
  Object.entries(data).filter(([key]) => ['id', 'name'].includes(key))
);

// Middleware bundle optimization
// middleware.ts
import { NextResponse } from 'next/server';
// ❌ Don't import heavy validation libraries
// import { z } from 'zod'; // 20kb+

// ✅ Use lightweight alternatives or native
const validateEmail = (email: string) => 
  /^[^\s@]+@[^\s@]+\.[^\s@]+$/.test(email);

// Split middleware logic
// middleware.ts (main entry)
export { authMiddleware as default } from './middleware/auth';
export const config = {
  matcher: ['/((?!_next/static|_next/image|favicon.ico).*)'],
};

// middleware/auth.ts (lightweight)
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function authMiddleware(request: NextRequest) {
  // Only JWT verification, no heavy logic
  return NextResponse.next();
}
```

### Memory and Execution Limits

Understanding edge constraints:

```typescript
/**
 * Edge Runtime Limits (Vercel):
 * - Execution: 50ms (Hobby), 30s (Pro/Enterprise)
 * - Memory: 128MB (Hobby), 1024MB (Pro)
 * - Request body: 4.5MB
 * - Response body: No limit (streaming supported)
 */

// Handling large responses with streaming
// app/api/large-data/route.ts
export const runtime = 'edge';

export async function GET() {
  const encoder = new TextEncoder();
  
  const stream = new ReadableStream({
    async start(controller) {
      // Stream data in chunks to minimize memory usage
      for (let i = 0; i < 1000; i++) {
        const chunk = JSON.stringify({ 
          id: i, 
          data: 'x'.repeat(1000) // 1KB per item
        }) + '\n';
        
        controller.enqueue(encoder.encode(chunk));
        
        // Yield to event loop every 100 items
        if (i % 100 === 0) {
          await new Promise(resolve => setTimeout(resolve, 0));
        }
      }
      
      controller.close();
    },
  });
  
  return new Response(stream, {
    headers: { 'Content-Type': 'application/x-ndjson' },
  });
}

// Handling CPU-intensive tasks
// Offload to background queue if >50ms expected
// app/api/compute/route.ts
export const runtime = 'edge';

export async function POST(request: Request) {
  const data = await request.json();
  
  // For heavy computation, enqueue to background worker
  if (data.complexity > 1000) {
    await fetch(`${process.env.WORKER_URL}/enqueue`, {
      method: 'POST',
      headers: { 'Authorization': `Bearer ${process.env.WORKER_SECRET}` },
      body: JSON.stringify(data),
    });
    
    return Response.json({ status: 'queued', id: data.id });
  }
  
  // Light computation can run at edge
  const result = lightComputation(data);
  return Response.json(result);
}
```

### Error Handling and Fallbacks

Resilience patterns for edge failures:

```typescript
// lib/edge/fallback.ts
export async function withFallback<T>(
  primary: () => Promise<T>,
  fallback: () => Promise<T>,
  timeoutMs: number = 5000
): Promise<T> {
  const timeout = new Promise<never>((_, reject) =>
    setTimeout(() => reject(new Error('Timeout')), timeoutMs)
  );
  
  try {
    return await Promise.race([primary(), timeout]);
  } catch (error) {
    console.warn('Primary failed, using fallback:', error);
    return fallback();
  }
}

// Usage
// app/api/critical/route.ts
export const runtime = 'edge';

export async function GET(request: Request) {
  const data = await withFallback(
    // Primary: Edge cache
    async () => {
      const cache = caches.default;
      const cached = await cache.match(request);
      if (!cached) throw new Error('Cache miss');
      return cached.json();
    },
    // Fallback: Origin database
    async () => {
      return fetchFromOriginDatabase();
    },
    100 // 100ms timeout for cache lookup
  );
  
  return Response.json(data);
}
```

## Key Takeaways from Chapter 49

1. **Edge Runtime Characteristics**: The Edge Runtime uses V8 isolates with sub-millisecond cold starts and global distribution across 100+ locations, but lacks Node.js APIs (fs, process) and native modules. Use it for latency-sensitive operations requiring geographical distribution, keep Node.js for heavy computation and stateful connections.

2. **Database Access**: Connect to databases from edge using HTTP-based drivers (Neon serverless, PlanetScale, Vercel Postgres) rather than TCP-based connections. Implement connection pooling via these drivers to prevent connection exhaustion, and consider read replicas for geographically distributed users.

3. **Middleware Optimization**: Execute authentication, A/B testing, and geographical routing in Middleware to prevent requests from reaching origin servers. Use JWT verification with `jose` (edge-compatible) instead of database sessions for zero-latency auth checks, and implement rate limiting using Upstash Redis for distributed counting.

4. **Caching Strategies**: Utilize the Cache API in edge functions for programmatic caching with fine-grained control. Implement stale-while-revalidate patterns (`s-maxage=60, stale-while-revalidate=3600`) to serve cached content instantly while refreshing in background, and use tag-based invalidation for cache purging.

5. **Geographical Intelligence**: Access user location via `request.geo` (country, city, region) to implement GDPR-compliant routing, currency detection, and language redirection. Route users to nearest database read replicas using middleware headers (`x-preferred-region`) to minimize query latency.

6. **AI at Edge**: Deploy lightweight ML inference (translation, sentiment analysis) at edge locations using Cloudflare AI or streaming OpenAI responses to reduce latency for global users. Use the Web Streams API to handle large responses without memory constraints, chunking data for progressive delivery.

7. **Resilience Patterns**: Implement timeouts and fallback strategies for edge functions—race against timeout promises, cache lookups before database queries, and offload CPU-intensive tasks (>50ms) to background queues. Monitor memory usage (128MB-1024MB limits) and avoid importing heavy libraries like full Lodash or Zod in middleware bundles.

## Coming Up Next

**Chapter 50: Serverless Architecture**

While edge computing excels at low-latency global distribution, complex business logic often requires the full power of serverless functions. In Chapter 50, we'll explore AWS Lambda and Vercel Functions patterns for Next.js, managing cold starts, optimizing function duration and memory allocation, implementing idempotency for payment processing, and architecting event-driven workflows using SNS/SQS. You'll learn when to choose edge versus serverless Node.js runtimes based on workload characteristics.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../9. Real_world_projects/48. social_media_platform.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='50. serverless_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
